In [1]:
import numpy as np
from IPython.display import Audio
from scipy.io.wavfile import write

In [2]:
def karplus_strong(wavetable, n_samples, stretch_factor):

    samples = []

    curr_sample = 0
    prev_value = 0

    while len(samples) < n_samples:

        stretch = np.random.binomial(1, 1 - 1/stretch_factor)
        if stretch == 0:
            wavetable[curr_sample] = 0.5 * (wavetable[curr_sample] + prev_value)
        samples.append(wavetable[curr_sample])
        prev_value = samples[-1]
        curr_sample = (curr_sample + 1) % wavetable.size

    return np.array(samples)

In [3]:
class GuitarString:
    def __init__(self, pitch, fs, A, S, T, noise_type):
        """Initialize Guitar String"""
        self.pitch = pitch                      # Frecuencia de la nota
        self.fs = fs                            # Frecuencia de Sampleo
        self.S = 1                              # Stretch Factor
        self.A = A                              # Amplitud
        self.T = T                              # Duración de la nota (samples)
        self.noise_type = noise_type            # Tipo de Ruido Inicial
        self.init_wavetable()
        self.init_samples()
        
    def init_wavetable(self):
        """Generate new Wavetable for String"""
        self.L = int(np.floor(self.fs / int(self.pitch)-1/2/self.S))
        if self.noise_type == "normal":
            self.wavetable = (self.A * np.random.normal(0,1,self.L)).astype(np.float32)
        if self.noise_type == "uniform":
            self.wavetable = (self.A * np.random.uniform(-1, 1, self.L)).astype(np.float32)
        if self.noise_type == "2-level":
            self.wavetable = (self.A * 2 * np.random.randint(0, 2, self.L) - 1).astype(np.float32)

    def init_samples(self):
        """Create sound samples for string"""
        self.samples = karplus_strong(self.wavetable, 2*self.T, self.S)
        
    def get_samples(self):
        """Return Sound Samples"""
        return self.samples

In [6]:
guitar = GuitarString(440, 44100, 0.5, 1, 44100/2, "normal")
note_one = guitar.get_samples()
guitar = GuitarString(440, 44100, 0.5, 1, 44100/2, "normal")
note_two = guitar.get_samples()
audio = np.concatenate((note_one, note_two))
audio = audio * (2**15 - 1) / np.max(np.abs(audio))
audio = audio.astype(np.int16)
Audio(audio, rate=44100)

In [5]:
write('concat.wav', 44100, audio)